
# ParTES plotting notebook

## V-3 Visualize for tool: meas_pair
### V-3.1 Single rank inverted CDF plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

folder = '.'
rank = 2
cut_tile = 0.99
# Choose y-axis scale, available: 'log10', 'log2', 'linear'
y_scale = 'linear'

pattern = os.path.join(folder, f'meas_pair_r{rank}_ng*.csv')
csv_files = glob.glob(pattern)

plt.figure(figsize=(10, 6))
for csv_file in csv_files:
    data = np.loadtxt(csv_file)
    sorted_data = np.sort(data)
    n = len(sorted_data)
    quantiles = np.arange(1, n + 1) / n
    
    # Apply y-axis scaling
    if y_scale == 'log10':
        sorted_data = np.log10(sorted_data)
    elif y_scale == 'log2':
        sorted_data = np.log2(sorted_data)
    # For 'linear', no transformation needed
    
    # Filter data to only include points where x <= cut_tile
    mask = quantiles <= cut_tile
    filt_q = quantiles[mask]
    filt_t = sorted_data[mask]
    
    nsub = os.path.basename(csv_file).split('_ng')[1].split('.csv')[0]
    plt.plot(filt_q, filt_t, label=f'nsub={nsub}', marker='o', markersize=3)

plt.xlabel('Quantile')
plt.ylabel('Time (ns)')
plt.title(f'ICDF Comparison for Rank {rank}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
